In [1]:

import os
import pty
import subprocess
import pprint
import time
import json
# https://github.com/helgefmi/ohno/blob/master/ohno/client/pty.py ## loosely based on this code
class Nethack:
    def __init__(self):
        self.master = None
        self.slave = None
        self.process = None

    def start_process(self):
        self.master, self.slave = pty.openpty()
        self.process = subprocess.Popen(
            ["nethack"],
            stdin=self.slave,
            stdout=self.slave,
            stderr=self.slave,
            close_fds=True, # this line comes from Bing Chat and I don't know what it does really
            universal_newlines=True,
        )
        os.close(self.slave) # Copilot added this so take it with a grain of salt...if it works, we don't need to save the reference

    def read_output(self, sleep = 0.25, chars = 8192):
    #def read_output(self, sleep = 0.25, chars = 8192, print_output = True):
        if self.process is None:
            raise ValueError("Process not started")
        if sleep is not None and sleep > 0:
            time.sleep(sleep)
        output = os.read(self.master, chars).decode()
        return output

    def send_command(self, command):
        #os.write(self.master, command.encode() + b"\n")
        os.write(self.master, command.encode())

    def close(self):
        try:
            self.process.terminate()
            os.close(self.master)
        except:
            pass

    @staticmethod
    def clean_locks():
        password = json.load(open("private.json"))["sudo"]
        os.system(f'echo "{password}" | sudo -S rm -f /var/games/nethack/*lock*')

# 24, 80

In [2]:
import stransi
import re

ansi_escape_codes = {
    r"\x1b7": "save cursor",
    r"\x1b\[\?47h": "set alternate screen",
    r"\x1b\)0": "set to ASCII",
    r"\x1b\[H": "move cursor to upper left",
    r"\x1b\[2J": "clear screen", # this might be a parameter?
    r"\x1b\[(\d+);(\d+)H": "move cursor",
    r"\x1b\[K": "clear to end of line",
    r"\x1b\[C": "move cursor right",
    r"\x1b8": "restore cursor",
}

class Terminal:
    unsupported = {
        r"\x1b7": "save cursor",
        r"\x1b8": "restore cursor",
        r"\x1b\[\?47h": "set alternate screen",
        r"\x1b\)0": "set to ASCII",
    }

    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.reset()

    def reset(self):
        self.cursor_row = 0
        self.cursor_col = 0
        self.characters = [[' ' for i in range(self.cols)] for _ in range(self.rows)]
        self.foregrounds = [["white" for i in range(self.cols)] for _ in range(self.rows)]
        self.backgrounds = [["black" for i in range(self.cols)] for _ in range(self.rows)] # does Nethack actually do this?

    def text(self):
        s = ""
        for i in range(self.rows):
            for j in range(self.cols): # let's ignore colors for now
                s += self.characters[i][j]
            s += "\n"
        return s
    
    def print(self):
        print(self.text())
                
    def update(self, input, print_instructions = False):
        #self.reset()
        # this is where the logic is going to get complex
        cleaned = input
        for k, v in Terminal.unsupported.items():
            # first we need to check if the key is in the input
            if re.search(k, cleaned):
                print(f"found unsupported {k} : {v}")
            rgx = re.compile(k)
            cleaned = re.sub(rgx, "", cleaned)
        ansi = stransi.Ansi(cleaned)
        for instr in ansi.instructions():
            if print_instructions: 
                print(instr) # for now
            if isinstance(instr, str):
                for i in range(len(instr)):
                    ch = instr[i]
                    if ch == "\r":
                        self.cursor_col = 0
                    elif ch == "\b":
                        self.cursor_col -= 1
                    elif ch == "\n":
                        self.cursor_row += 1
                    elif ch == '\x0f' or ch == '\x00':
                        pass
                    else:
                        try:
                            self.characters[self.cursor_row][self.cursor_col] = ch
                            self.cursor_col += 1
                        except:
                            print(f"something out of bounds: col={self.cursor_col}, row={self.cursor_row}, i={i} len={len(instr)}")
            elif isinstance(instr, stransi.SetCursor):
                if instr.move.relative:
                    self.cursor_row += instr.move.x
                    self.cursor_col += instr.move.y
                else:
                    self.cursor_row = instr.move.x
                    self.cursor_col = instr.move.y 
            elif isinstance(instr, stransi.SetClear):
                if instr.region == stransi.clear.Clear.SCREEN:
                    self.reset()
                elif instr.region == stransi.clear.Clear.LINE_AFTER:
                    for i in range(self.cursor_col, self.cols):
                        self.characters[self.cursor_row][i] = ' '
                elif instr.region == stransi.clear.Clear.LINE:
                    print("stransi.clear.Clear.LINE, was not expecting this.")
                    for i in range(self.cols):
                        self.characters[self.cursor_row][i] = ' '
                elif instr.region == stransi.clear.Clear.LINE_BEFORE:
                    print("stransi.clear.Clear.LINE_BEFORE, was not expecting this.")
                    for i in range(0, self.cursor_col):
                        self.characters[self.cursor_row][i] = ' '
                elif instr.region == stransi.clear.Clear.SCREEN_AFTER:
                    print("stransi.clear.Clear.SCREEN_AFTER, was not expecting this.")
                    for i in range(self.cursor_row, self.rows):
                        for j in range(self.cols):
                            self.characters[i][j] = ' '
                elif instr.region == stransi.clear.Clear.SCREEN_BEFORE:
                    print("stransi.clear.Clear.SCREEN_BEFORE, was not expecting this.")
                    for i in range(0, self.cursor_row):
                        for j in range(self.cols):
                            self.characters[i][j] = ' '
                else:
                    print(instr)
                    print(instr.region)
                    print("I guess I didn't handle all the clears")
            elif isinstance(instr, stransi.SetColor):
                if instr.role == stransi.color.ColorRole.FOREGROUND:
                    self.foregrounds[self.cursor_row][self.cursor_col] = instr.color.ansi256.code
                elif instr.role == stransi.color.ColorRole.BACKGROUND:
                    self.backgrounds[self.cursor_row][self.cursor_col] = instr.color.ansi256.code
                else:
                    print("something other kind of color instruction?")
                    print(instr)
            elif isinstance(instr, stransi.SetAttribute):
                if instr.attribute == stransi.attribute.Attribute.NORMAL:
                    pass
                    #print("normal")
                elif instr.attribute == stransi.attribute.Attribute.BOLD:
                    pass
                    #print("bold")
                elif instr.attribute == stransi.attribute.Attribute.REVERSE:
                    pass
                    #print("reverse")
                else:
                    print("another attribute?")
                    print(instr)
            else:
                if print_instructions:
                    print("^^^ this instruction was not handled.")
        # this is how the version I'm drawing from did it
        if self.cursor_col >= self.cols:
            self.cursor_col = self.cursor_col - self.cols
            self.cursor_row += 1
        if self.cursor_row >= self.rows:
            self.cursor_row = self.rows - 1

In [3]:
Nethack.clean_locks()
nethack = Nethack()
nethack.start_process()
text = nethack.read_output() # setting chars = 1 indeed fetches just 1.  100 is not enough. ~400 is more than enough.  You can ask for way too many and it won't freeze up unless you ask for another read.
term = Terminal(24, 80)
term.update(text)
print(term.text())
# you need to do # quit and then a bunch fo confirmations in order to actually quit.
# n takes you to a screen to select stuff.
# Sending "yn" at the first opportunity seems to suggest that it did two separate screens.
# There are a couple of complications here

[sudo] password for perplexity: 

found unsupported \x1b7 : save cursor
found unsupported \x1b\[\?47h : set alternate screen
found unsupported \x1b\)0 : set to ASCII
                                                                                
NetHack, Copyright 1985-2020                                                    
         By Stichting Mathematisch Centrum and M. Stephenson.                   
         Version 3.6.6 Unix, revised Feb 25 14:00:45 2021.                      
         See license for details.                                               
                                                                                
                                                                                
Shall I pick character's race, role, gender and alignment for you? [ynaq]       
                                                                                
                                                                                
                                                          

In [4]:
def next_command(c, verbose = False):
    nethack.send_command(c)
    text = nethack.read_output()
    if verbose:
        print(text)
    term.update(text, print_instructions = verbose)
    print(term.text())
    return text

In [5]:
text = next_command("y")

       name: perplexity                  Is this ok? [ynq]                      
       role: Priest                                                             
       race: elf                         perplexity, chaotic elven Priest       
     gender: male                                                               
  alignment: chaotic                     y + Yes; start game                    
                                         n - No; choose role again              
                                         q - Quit                               
                                                                                
                                        (end)                                   
                                                                                
                                                                                
                                                                                
                            

In [6]:
text = next_command("y")

            It is written in the Book of Manannan Mac Lir:                      
                                                                                
                After the Creation, the cruel god Moloch rebelled               
                against the authority of Marduk the Creator.                    
                Moloch stole from Marduk the most powerful of all               
                the artifacts of the gods, the Amulet of Yendor,                
                and he hid it in the dark cavities of Gehennom, the             
                Under World, where he now lurks, and bides his time.            
                                                                                
            Your god Manannan Mac Lir seeks to possess the Amulet, and with it  
            to gain deserved ascendance over the other gods.                    
                                                                                
            You, a newly tra

In [7]:
text = next_command(" ") ## We are sometimes getting out of bounds col=80, which probably shouldn't happen?  

Hello perplexity, welcome to NetHack!  You are a chaotic elven Priest.          
                                                                                
                                                                                
                                                                                
                    -----------                                                 
                    |....@$d..|                                                 
                    |.....f....                                                 
                    ------.----                                                 
                                                                                
                                                                                
                                                                                
                                                                                
                            

In [11]:
# h is left, j is down, k is up, l is right
text = next_command("l")  # okay, something odd is happening here: When I move, the player cursor disappears, and then the right way moves to the left.  So it seems like some portion of the text is getting blotted out.
# interesting, that final character is \x08 which is backspace.  I wonder if that has smething to do with it.

                                                                                
                                                                                
                                                                                
                                                                                
                    -----------                                                 
                    |....<$f..|                                                 
                    |.....@....                                                 
                    ------.----                                                 
                                                                                
                                                                                
                                                                                
                                                                                
                            

In [20]:
print(term.text())

                                                                                
                                                                                
                                                                                
                                                                                
                           -----                                                
                           -...|                                                
                           |..{|   #                                            
                           |..<.#@#                                             
                         ...----##@##                                           
                                                                                
                                                                                
                                                                                
                            

In [15]:
text

'\x1b[H\x1b[K\x1b[8;72H<\x1b[m\x1b[9;72H\x1b\x1b@\x1b[m\x08'

nethack.send_command("y")
text = nethack.read_output()
term.update(text, print_instructions = True)
print(term.text())

In [7]:
nethack.send_command(" ")
text = nethack.read_output()
term.update(text, print_instructions = True)
print(term.text())


SetCursor(move=CursorMove(x=0, y=-1, relative=True))
SetClear(region=<Clear.LINE_AFTER: 3>)
SetCursor(move=CursorMove(x=1, y=0, relative=False))
SetClear(region=<Clear.LINE_AFTER: 3>)
SetCursor(move=CursorMove(x=0, y=0, relative=False))
SetCursor(move=CursorMove(x=22, y=0, relative=False))
[
SetAttribute(attribute=<Attribute.REVERSE: 7>)
^^^ this instruction was not handled.
Perplexity the Plunderess     
SetAttribute(attribute=<Attribute.NORMAL: 0>)
^^^ this instruction was not handled.
] St:17 Dx:16 Co:18 In:7 Wi:9 Ch:8 Chaotic
SetCursor(move=CursorMove(x=23, y=0, relative=False))
Dlvl:1 $:0 HP:16(16) Pw:2(2) AC:7 Xp:1
SetCursor(move=CursorMove(x=9, y=55, relative=False))
                                                                                
                                                                                
                                                                                
                                                                         

In [ ]:
#unhandled things: 
#SetAttribute(attribute=<Attribute.NORMAL: 0>)
#SetAttribute(attribute=<Attribute.BOLD: 1>)
#SetAttribute(attribute=<Attribute.REVERSE: 7>)
#SetCursor(move=CursorMove(x=0, y=-1, relative=True))

In [9]:
# h is left, j is down, k is up, l is right
nethack.send_command("h")
text = nethack.read_output()
term.update(text, print_instructions = True)
print(term.text())
# '\x1b[H\x1b[K\x1b[17;70H\x1b[1m\x1b[37m@\x1b[m<\x1b[m\x08\x08'

SetCursor(move=CursorMove(x=0, y=0, relative=False))
The door opens.
SetClear(region=<Clear.LINE_AFTER: 3>)
SetCursor(move=CursorMove(x=10, y=54, relative=False))
SetColor(role=<ColorRole.FOREGROUND: 30>, color=Ansi256(3))
^^^ this instruction was not handled.
-
SetAttribute(attribute=<Attribute.NORMAL: 0>)
^^^ this instruction was not handled.
SetCursor(move=CursorMove(x=0, y=17, relative=False))
Rex misses the jackal.
SetClear(region=<Clear.LINE_AFTER: 3>)
SetCursor(move=CursorMove(x=10, y=55, relative=False))
SetCursor(move=CursorMove(x=0, y=41, relative=False))
The jackal bites!
SetClear(region=<Clear.LINE_AFTER: 3>)
SetCursor(move=CursorMove(x=22, y=0, relative=False))
[
SetAttribute(attribute=<Attribute.REVERSE: 7>)
^^^ this instruction was not handled.
Perplexity the Plunderess 
SetAttribute(attribute=<Attribute.NORMAL: 0>)
^^^ this instruction was not handled.
    ]
SetCursor(move=CursorMove(x=23, y=0, relative=False))
SetCursor(move=CursorMove(x=23, y=10, relative=False))
 HP:

In [8]:
text

'\x1b[H\x1b[K\x1b[19;51H\x1b\x1b@\x1b[m<\x1b[m\x08\x08'